
---

# Transformaciones
---


In [3]:
import pandas as pd
import ast
df = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\movies_dataset.csv', low_memory=False)
df2 = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\credits.csv', low_memory=False)

#'''trabajaré al mismo tiempo con el DF Credits, ya que algunas funciones de la API necesitan datos provenientes de este'''

In [4]:
#los convertimos a 'parquet' para optimizar el rendimineto
df.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')
# Guardar con compresión snappy
df2.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')



In [5]:
# Contar el número de filas duplicadas
duplicados = df.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados

print("Número de filas duplicadas eliminadas:", numero_de_duplicados)

#segundo DF
# Contar el número de filas duplicadas 
duplicados = df2.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df2.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados
print("Número de filas duplicadas eliminadas en el segundo DF:", numero_de_duplicados)

Número de filas duplicadas eliminadas: 17
Número de filas duplicadas eliminadas en el segundo DF: 37


In [6]:

'''Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.'''
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])


In [7]:
'''Los valores nulos del campo release date deben eliminarse.'''

df = df.dropna(subset=['release_date'])

In [8]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.'''

df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [9]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

#convertimos las fechas al datatype datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 
df['release_year'] = df['release_date'].dt.year

In [10]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''
# Convertir las columnas 'revenue' y 'budget' a tipo numérico
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
# Rellenar valores nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando revenue / budget
# Usar np.where para evitar división por 0
import numpy as np
df['return'] = np.where(df['budget'] == 0, 0, df['revenue'] / df['budget'])

In [12]:
''' Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una
lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API!
O bien buscar la manera de acceder a esos datos sin desanidarlos.'''
# Función para convertir cadenas a diccionarios o listas
def convertir_en_obj(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x  # Retorna la cadena si no es un diccionario válido
    return x

# Aplicar la conversión a las series 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_en_obj)
df['genres'] = df['genres'].apply(convertir_en_obj)
df['production_companies'] = df['production_companies'].apply(convertir_en_obj)
df['production_countries'] = df['production_countries'].apply(convertir_en_obj)
df['spoken_languages'] = df['spoken_languages'].apply(convertir_en_obj)

# Desanidar diccionarios
BTC_df = df['belongs_to_collection'].apply(pd.Series)
BTC_df = BTC_df.drop(columns = 0, errors='ignore')  # Eliminar columna de índice si existe
nuevos_nombres_BTC = ['BTCid', 'BTCname', 'BTCposter_path', 'BTCbackdrop_path']
BTC_df.columns = nuevos_nombres_BTC

# Desanidar listas
genres_df = df['genres'].apply(pd.Series)
production_companies_df = df['production_companies'].apply(pd.Series)
production_countries_df = df['production_countries'].apply(pd.Series)
spoken_languages_df = df['spoken_languages'].apply(pd.Series)

# Asegurarse de que no haya nombres de columnas duplicados
genres_df = genres_df.rename(columns=lambda x: f'genre_{x}')
production_companies_df = production_companies_df.rename(columns=lambda x: f'company_{x}')
production_countries_df = production_countries_df.rename(columns=lambda x: f'country_{x}')
spoken_languages_df = spoken_languages_df.rename(columns=lambda x: f'language_{x}')

# Eliminar las columnas originales
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'])

# Unir los DataFrames desanidado al DataFrame original
df = df.join(BTC_df)
df = df.join(production_companies_df)
df = df.join(production_countries_df)
df = df.join(spoken_languages_df)


C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:21: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  BTC_df = df['belongs_to_collection'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:27: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  genres_df = df['genres'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:28: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  production_companies_df = df['production_companies'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:29: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series


---

### Transformaciones sobre credits (API)
---


In [13]:
''' Al ser un DF muy grande y necesitar solo partes muy especificas de los diccionarios que componen las series del DF,
en vez de desanidarlo todo, utilizaremos otras tecnicas para extraer los datos que necesitamos, y con estos crearemos una nueva tabla,
esta misma contendra solamente los datos que necesitamos para las funciones de nuestra API '''

# Extraer los nombres de la columna 'cast'
names = df2['cast'].apply(lambda x: [item['name'] for item in ast.literal_eval(x) if 'name' in item])

# Convertir la serie 'names' en un DataFrame
names_df = pd.DataFrame(names.tolist())

# Añadir la palabra 'nombre' delante del nombre de cada serie
names_df.columns = ['nombre_' + str(col) for col in names_df.columns]

# Crear una serie que contenga únicamente el nombre del director
names_df['Nombre_Director'] = df2['crew'].apply(
    lambda x: next((item['name'] for item in ast.literal_eval(x) if item.get('job') == 'Director'), None)
)

# volvemos a añadir los ID a las filas, ya que los necesitaremos luego para hacer un "Join"
names_df['id'] = df2['id'].astype(int)





In [14]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')

In [54]:
names_df.head(1)

,nombre_0,nombre_1,nombre_2,nombre_3,nombre_4,nombre_5,nombre_6,nombre_7,nombre_8,nombre_9,...,nombre_305,nombre_306,nombre_307,nombre_308,nombre_309,nombre_310,nombre_311,nombre_312,Nombre_Director,id
0,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,John Ratzenberger,Annie Potts,John Morris,Erik von Detten,Laurie Metcalf,...,None,None,None,None,None,None,None,None,John Lasseter,862


#estas transformaciones las hago ya habiendo casi finalizado el trabajo, para el sistema de recomendación
#He decidido que para este crearé un nuevo DF, el cual será una versión reducida del mismo, y que mantendrá unicamente las series que considere pertinentes

---

### Exportación de los Datafames

---


In [63]:
df.to_parquet('transformados.parquet', index=False)


In [ ]:
df2


,nombre_0,nombre_1,nombre_2,nombre_3,nombre_4,nombre_5,nombre_6,nombre_7,nombre_8,nombre_9,...,nombre_310,nombre_311,nombre_312,Nombre_Director,id,return,title,release_year,budget,revenue
0,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,John Ratzenberger,Annie Potts,John Morris,Erik von Detten,Laurie Metcalf,...,,,,John Lasseter,862,12.451801,Toy Story,1995.0,30000000.0,373554033.0
1,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,Bonnie Hunt,Bebe Neuwirth,David Alan Grier,Patricia Clarkson,Adam Hann-Byrd,Laura Bell Bundy,...,,,,Joe Johnston,8844,4.043035,Jumanji,1995.0,65000000.0,262797249.0
2,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,Daryl Hannah,Burgess Meredith,Kevin Pollak,,,,...,,,,Howard Deutch,15602,0.000000,Grumpier Old Men,1995.0,0.0,0.0
3,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon,Gregory Hines,Dennis Haysbert,Michael Beach,Mykelti Williamson,Lamont Johnson,Wesley Snipes,...,,,,Forest Whitaker,31357,5.090760,Waiting to Exhale,1995.0,16000000.0,81452156.0
4,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley,George Newbern,Kieran Culkin,BD Wong,Peter Michael Goetz,Kate McGregor-Stewart,Jane Adams,...,,,,Charles Shyer,11862,0.000000,Father of the Bride Part II,1995.0,0.0,76578911.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,Leila Hatami,Kourosh Tahami,Elham Korda,,,,,,,,...,,,,Hamid Nematollah,439050,NaN,NaN,NaN,NaN,NaN
45534,Angel Aquino,Perry Dizon,Hazel Orencio,Joel Torre,Bart Guingona,Soliman Cruz,Roeder,Angeli Bayani,Dante Perez,Betty Uy-Regala,...,,,,Lav Diaz,111109,0.000000,Century of Birthing,2011.0,0.0,0.0
45535,Erika Eleniak,Adam Baldwin,Julie du Page,James Remar,Damian Chapa,Louis Mandylor,Tom Wright,Jeremy Lelliott,James Quattrochi,Jason Widener,...,,,,Mark L. Lester,67758,0.000000,Betrayal,2003.0,0.0,0.0
45536,Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlov,Aleksandr Chabrov,Vera Orlova,,,,,,...,,,,Yakov Protazanov,227506,0.000000,Satan Triumphant,1917.0,0.0,0.0
